<a href="https://colab.research.google.com/github/nosferatu0205/BanglaSpellCheck/blob/main/AI_Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import functools
import re
import string


In [ ]:
rule_words = ['ই', 'ও', 'তো', 'কে', 'তে', 'রা', 'চ্ছি', 'চ্ছিল', 'চ্ছে', 'চ্ছিস', 'চ্ছিলেন', 'চ্ছ', 'য়েছে', 'েছ', 'েছে',
              'েছেন', 'রছ', 'রব', 'েল', 'েলো', 'ওয়া', 'েয়ে', 'য়', 'য়ে', 'েয়েছিল', 'েছিল', 'য়েছিল', 'েয়েছিলেন',
              'ে.েছিলেন', 'েছিলেন', 'লেন', 'দের', 'ে.ে', 'ের', 'ার', 'েন', 'বেন', 'িস', 'ছিস', 'ছিলি', 'ছি', 'ছে', 'লি',
              'বি', 'ে', 'টি', 'টির', 'েরটা', 'েরটার', 'টা', 'টার', 'গুলো', 'গুলোর', 'েরগুলো', 'েরগুলোর'
              'যোগ্য','কেই','েও','সহ','রা','ভাবে','কারি','কৃত','ই','কে','র','কি','েই','ভাবে','গুলো',
              'তে','েতে','গন','মুলক','সুচক','টুকু','টুকুই','গুলি','পদ','সমুহ','সংক্রান্ত','সংলগ্ন','সংশ্লিষ্ট',
              'সুত্রে','রুপে','ানুসারে','ানুযায়ি','তত্ত্','ি','মুখি','প্রতি','ভাবে','য়'
              ]
rule_dict = {"রছ":"র","রব":"র","েয়ে":"া","েয়েছিল":"া","েয়েছিলেন":"া","ে.েছিলেন":"া.","ে.ে":"া."}



In [ ]:
bn2en={'0985':'a','0986':'aa','0987':'i','0988':'ii',
           '0989':'u','098A':'uu','098B':'r','09E0':'rr',
           '098C':'l','09E1':'ll','098F':'e','0990':'ai',
           '0993':'o','0994':'au',
           '0995':'ka','0996':'kha','0997':'ga','0998':'gha',
           '0999':'nga','099A':'ca','099B':'cha','099C':'ja',
           '099D':'jha','099E':'nya','099F':'tta', '09A0':'ttha',
           '09A1':'dda','09DC':'rra','09A2':'ddha','09DD':'rha',
           '09A3':'nna','09A4':'ta','09A5':'tha','09A6':'da',
           '09A7':'dha','09A8':'na','09AA':'pa','09AB':'pha',
           '09AC':'ba','09AD':'bha','09AE':'ma','09AF':'ya',
           '09DF':'yya','09B0':'ra','09B2':'la','09B6':'sha',
           '09B7':'ssa','09B8':'sa','09B9':'ha','09CE':'ta',
           '09BE':'aa','09BF':'i','09C0':'ii','09C1':'u',
           '09C2':'uu','09C3':'r','09C4':'rr','09C7':'e',
           '09C8':'ai', '09CB':'o','09CC':'au',
           '09CD':'','09D7':'','09E6':'0',
           '09E7':'1','09E8':'2','09E9':'3','09EA':'4',
           '09EB':'5','09EC':'6','09ED':'7','09EE':'8',
           '09EF':'9',
           '09BC':'','0982':'n','0983':'','0981':''
           }
bn2enNum={
        '09E6': '0',
        '09E7': '1', '09E8': '2', '09E9': '3', '09EA': '4',
        '09EB': '5', '09EC': '6', '09ED': '7', '09EE': '8',
        '09EF': '9'
    }
bn2enPunc={
        '09BE': 'a', '09BF': 'i', '09C0': 'i', '09C1': 'u',
        '09C2': 'u', '09C3': 'r', '09C4': 'r', '09C7': 'e',
        '09C8': 'ai', '09CB': 'o', '09CC': 'au','09B0':'ra'
    }
bn2enSuffix={
        '09BE': 'a', '09B0':'ra','09CB': 'o','0993':'o','09C7': 'e'
    }
bn_norm={
        '09C0':'\u09bf','09C2':'\u09c1','09C4':'\u09c3','09A3':'\u09a8','0988':'\u0987','098A':'\u0989'
    }
bn_serial={
        '0985': 0, '0986': 1, '09BE': 2, '0987': 3, '09BF': 4,'0988': 5, '09C0': 6,
        '0989': 7, '09C1': 8, '098A': 9, '09C2': 10,'098B': 11,'09C3': 12, '09E0': 13,'09C4': 14,
        '098C': 15, '09E1': 16, '098F': 17,  '09C7': 18, '0990': 19,'09C8': 20,
        '0993': 21, '09CB': 22, '0994': 23, '09CC': 24,
        '0995': 25, '0996': 26, '0997': 27, '0998': 28,
        '0999': 29, '099A': 30, '099B': 31, '099C': 32,
        '099D': 33, '099E': 34, '099F': 35, '09A0': 36,
        '09A1': 37, '09DC': 38, '09A2': 39, '09DD': 40,
        '09A3': 41, '09A4': 42, '09A5': 43, '09A6': 44,
        '09A7': 45, '09A8': 46, '09AA': 47, '09AB': 48,
        '09AC': 49, '09AD': 50, '09AE': 51, '09AF': 52,
        '09DF': 53, '09B0': 54, '09B2': 55, '09B6': 56,
        '09B7': 57, '09B8': 58, '09B9': 59, '09CE': 60,
        '09E6': 61,
        '09E7': 62, '09E8': 63, '09E9': 64, '09EA': 65,
        '09EB': 66, '09EC': 67, '09ED': 68, '09EE': 69,
        '09EF': 70,
        '09CD': 71, '09D7': 72,
        '09BC': 73, '0982': 74, '0983': 75, '0981': 76
      }

In [ ]:
def dust_removal(word):
        s=""
        for c in word:
            g = c.encode("unicode_escape")
            g = g.upper()
            g = g[2:]
            g = g.decode('utf-8')
            if g in bn2en:
                s+=c
        return s

In [ ]:
word_vec=[]
word_dict={}
word_dict2={}

In [ ]:
def init_dictionary():
    try:
        with open("/content/word2.txt", "r", encoding='utf-8') as file1:
            for word in file1:
                word = word.replace('\n', '')
                word_vec.append(word)
                word_dict[word] = 1
    except UnicodeDecodeError:
        with open("/content/word2.txt", "r", encoding='latin-1') as file1:
            for word in file1:
                word = word.replace('\n', '')
                word_vec.append(word)
                word_dict[word] = 1

    try:
        with open("/content/word3.txt", "r", encoding='utf-8') as file2:
            for word in file2:
                word = word.replace('\n', '')
                word_dict2[word] = 1
    except UnicodeDecodeError:
        with open("/content/word3.txt", "r", encoding='latin-1') as file2:
            for word in file2:
                word = word.replace('\n', '')
                word_dict2[word] = 1


In [ ]:
def word_normalize(word):
        s = ""
        for c in word:
            g = c.encode("unicode_escape")
            g = g.upper()
            g = g[2:]
            g = g.decode('utf-8')
            if g in bn_norm:
                g = bn_norm[g].encode().decode('utf-8')
                s+=g
                continue
            s+=c
        return s

In [ ]:
def search(word):
        if word_normalize(word) in word_dict:
            return True
        return False

In [ ]:



def normalize_tokenizer(sent):
    sent=word_normalize(sent)
    tokens=sent.split()
    temp_tokens=[]
    for i in tokens:
        if len(dust_removal(i))==0:
            continue
        i=dust_removal(i)
        temp_tokens.append(i)
    return temp_tokens


In [ ]:
init_dictionary()

In [ ]:
dust_removal("অবস....")

'অবস'

In [ ]:
search("অবসবরপ্র")

False

In [ ]:
normalize_tokenizer("অবস অবসর")

['অবস', 'অবসর']